<a href="https://colab.research.google.com/github/sjay8/Project/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pre-Semester Project- Sana Jayaswal 

In [2]:
#after the data set was uploaded into the files tab 

import io
import numpy as np
import pandas as pd
import re
# plotting
import seaborn as sns
from wordcloud import WordCloud
# nltk
from nltk.stem import WordNetLemmatizer
# sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report



In [81]:
data = pd.read_csv("data.csv")

In [4]:
!pip install keras

In [82]:
### building data intution 
print('length of data is', len(data))
print('Count of columns in the data is:  ', len(data.columns))
print('Count of rows in the data is:  ', len(data))

# how many unique values are there in valence?
print(data['valence'].unique())
data['valence'].nunique()

#working with only relevant parts of data
data = data.drop('author', 1)
#data=data[['valence','tweet']]
data.tail(5) #checking if above worked 


length of data is 1600000
Count of columns in the data is:   3
Count of rows in the data is:   1600000
[0 4]


,valence,tweet
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...
1599999,4,happy #charitytuesday @theNSPCC @SparksCharity...


In [83]:
data_pos = data[data['valence'] == 4]
data_neg = data[data['valence'] == 0]

data_pos = data_pos.iloc[:int(20000)]
data_neg = data_neg.iloc[:int(20000)]
data= pd.concat([data_pos, data_neg])
data['tweet']=data['tweet'].str.lower()
data ['tweet'].head()

800000         i love @health4uandpets u guys r the best!! 
800001    im meeting up with one of my besties tonight! ...
800002    @darealsunisakim thanks for the twitter add, s...
800003    being sick can be really cheap when it hurts t...
800004      @lovesbrooklyn2 he has that effect on everyone 
Name: tweet, dtype: object

In [113]:
label_to_sentiment = {0:"Negative", 4:"Positive"}
def mapper(label):
     return label_to_sentiment[label]
data.valence = data.valence.apply(lambda x: mapper(x))

In [84]:
stopwords = ['a', 'about','like', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can',"can't", 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from',
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i','im', 'if', 'in',
             'into','is', 'it', 'its',"it's","i'm",  'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're','s', 'same', 'she', "she's", 'should', "should've",'so', 'some', 'such',
             't', 'than', 'that', "that'll","that's", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those',
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "you'd","you'll", "you're",
             "you've", 'your', 'yours', 'yourself', 'yourselves']

In [85]:
data['tweet']= data['tweet'].str.lower()
def clean_symbols(data):
    data= re.sub('https?://[A-Za-z0-9./]+',' ',data)
    data= re.sub('@[A-Za-z0-9]+',' ', data)
    data= re.sub('#[A-za-z0-9]',' ', data) # remove #s
    data = re.sub('[()!%.;,?]', ' ', data)
    data = re.sub('[0-9]',' ', data)
    data = data.split()
    data = [w for w in data if not w in stopwords]
    data = " ".join(word for word in data)
    return data
data['tweet'] = data['tweet'].apply(lambda x: clean_symbols(x))




In [114]:
data.head()

,valence,tweet
800000,Positive,love u guys r best
800001,Positive,meeting one besties tonight cant wait - girl talk
800002,Positive,thanks twitter add sunisa got meet hin show dc...
800003,Positive,sick really cheap hurts much eat real food plu...
800004,Positive,effect everyone


In [ ]:
from keras.preprocessing.text import Tokenizer
#Tokenizing 
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data.tweet)
word_index = tokenizer.word_index
print(word_index)
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size :", vocab_size)

#Padding 
from keras.preprocessing.sequence import pad_sequences
X_train = pad_sequences(tokenizer.texts_to_sequences(data.tweet),maxlen = 30)
X_test = pad_sequences(tokenizer.texts_to_sequences(data.tweet),maxlen = 30)

In [120]:
labels = ['Negative', 'Positive']
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(data.valence.to_list())
y_train = encoder.transform(data.valence.to_list())
y_test = encoder.transform(data.valence.to_list())
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)


In [149]:
from tensorflow.keras.layers import Conv1D, Bidirectional, LSTM, Dense, Input, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.callbacks import ModelCheckpoint

model=Sequential()
model.add(Embedding(vocab_size,300,input_length=30,trainable=False))
# Input layer 

model.add(Input(shape=(30,), dtype='int32'))

# Passed on to the LSTM layer
model.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))

# Output Layer
model.add(Dense(1, activation='sigmoid'))




In [148]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_20 (Embedding)    (None, 30, 300)           9440100   
                                                                 
 input_5 (InputLayer)        multiple                  0         
                                                                 
 bidirectional_3 (Bidirectio  (None, 128)              186880    
 nal)                                                            
                                                                 
 dense_26 (Dense)            (None, 512)               66048     
                                                                 
 dropout_20 (Dropout)        (None, 512)               0         
                                                                 
 dense_27 (Dense)            (None, 512)               262656    
                                                     

In [130]:
training = model.fit(X_train, y_train, batch_size=64, epochs=8,
                    validation_data=(X_test, y_test))

Epoch 1/8
625/625 [==============================] - 83s 132ms/step - loss: 0.6251 - accuracy: 0.6496 - val_loss: 0.5659 - val_accuracy: 0.7121
Epoch 2/8
625/625 [==============================] - 83s 133ms/step - loss: 0.5762 - accuracy: 0.6960 - val_loss: 0.5100 - val_accuracy: 0.7387
Epoch 3/8
625/625 [==============================] - 83s 133ms/step - loss: 0.5270 - accuracy: 0.7340 - val_loss: 0.4169 - val_accuracy: 0.8231
Epoch 4/8
625/625 [==============================] - 84s 135ms/step - loss: 0.4801 - accuracy: 0.7666 - val_loss: 0.3748 - val_accuracy: 0.8475
Epoch 5/8
625/625 [==============================] - 86s 137ms/step - loss: 0.4408 - accuracy: 0.7919 - val_loss: 0.3154 - val_accuracy: 0.8897
Epoch 6/8
625/625 [==============================] - 86s 137ms/step - loss: 0.4042 - accuracy: 0.8129 - val_loss: 0.3014 - val_accuracy: 0.8921
Epoch 7/8
625/625 [==============================] - 86s 138ms/step - loss: 0.3671 - accuracy: 0.8345 - val_loss: 0.2383 - val_accuracy:

In [131]:
def predict_tweet_sentiment(score):
    return "Positive" if score>0.5 else "Negative"
scores = model.predict(X_test, verbose=1, batch_size=10000)
model_predictions = [predict_tweet_sentiment(score) for score in scores]

4/4 [==============================] - 10s 2s/step


In [133]:
from sklearn.metrics import classification_report
print(classification_report(list(data.valence), model_predictions))

              precision    recall  f1-score   support

    Negative       0.95      0.91      0.93     20000
    Positive       0.92      0.95      0.93     20000

    accuracy                           0.93     40000
   macro avg       0.93      0.93      0.93     40000
weighted avg       0.93      0.93      0.93     40000

